# RAG Evaluation using Amazon Bedrock Evaluation
This notebook demonstrates how you can very quickly build a basic evaluation suite using syntethic data generation, then grow from there.  
We pick RAG as the task to evaluate. The RAG is implemented using Bedrock Knowledge Base.
The evaluation software is Bedrock Evaluations using LLM-as-a-judge over RAG generation.  
**Note: This notebook was tested using the us-east-1 region.**

### Workshop Flow
Here are the main steps of this RAG Evaluation Workshop:

1. **Choose Your Dataset** - Select the business data you want to benchmark your RAG system with.

2. **Create Bedrock Knowledge Base** - Upload your document to S3 and create a knowledge base pointing to it.

3. **Generate Synthetic Evaluation Set** - Use Bedrock Playground with Sonnet 3.7 to create Q&A pairs from your document.

4. **Improve Evaluation Set & Upload to S3** - Review the synthetic data, make improvements, and upload to S3.

5. **Sync & Verify Knowledge Base** - Ensure your knowledge base has properly indexed the documents.

6. **Run RAG Evaluation** - Use Bedrock Evaluations to test different models and configurations.

7. **Compare Evaluation Results** - Analyze the performance data to determine the best approach.

#### 💡 See a [step-by-step walkthrough of this notebook in YouTube](https://youtu.be/aheBSDPfBqs)

## 🏁 Let's start! 🏁
## Choose your dataset
Consider which data you would like to benchmark over. It can be your own business data, or something like [חוק ביטוח אבטלה](https://www.btl.gov.il/Laws1/00_0001_000007.pdf) (In Hebrew). 


## Enable access to all models
1. In the console navigate to [Amazon Bedrock] > [[Model access]](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/modelaccess) to accept EULAs. Click [Modify model access]. Check all models (top checkbox), and click next.

## Create a Bedrock knowledge base
With the document at hand:
1. Copy the document to S3. e.g: `s3://my-bucket/bituah-leumi/dataset/00_0001_000007.pdf`
(use the S3 in the web console. Create a bucket if you see none).
2. Create a default knowledge base with this S3 folder path as input.
3. The knowledge base is being created. Continue to next step.

## Generate a syntethic evaluation set
Now we'll generate a 10 Q&A synthetic dataset that will later use to benchmark the knowledge base.
1. In the console navigate to [Amazon Bedrock] > [Chat / Text playground] and choose Sonnet 3.7.
2. In the system prompt add instructions that asks the model to generate Q&A and output in the format that RAG evaluation [expects for RAG generation](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-evaluation-prompt-retrieve-generate.html). Example prompt [here](./prompts/system_prompt_generate_qna_for_br_rag_eval.txt). 
3. Set temperature to 0. Set Response length to 32000.
4. Click the clip icon to upload the document as an attachment to the chat.
5. Write "Attached" in the prompt area.
6. Click [Run].
7. Watch your dataset being generated...
8. Validate the json: Copy the dataset and validate the jsonl with a tool like [https://jsonltools.com/jsonl-validator](https://jsonltools.com/jsonl-validator). Remove any prefix and suffix text added by the generation model. Click validate. If it reports an error in a given line, it's probably due to escaping. Try remove that line or the line before it, and retry to validate the json (it's okay to through aware data). 
9. Copy the lines to a new files and save it as `eval.jsonl`.

## Improve the syntethic evaluation set and upload it to S3.
We used a frontier model to geneate the Q&A which saved us a lot of time. Having said that, to be able to truly trust the evaluation, as domain expert you now need to review the Q&A and approch each Q&A pair. Fix or delete it. 
1. Review, approve/fix/delete Q&A pairs. and save the file.
2. Upload the file to S3. For example: `s3://my-bucket/bituah-leumi/eval/eval.jsonl`. Note it's S3 URI (will be used later).

## Sync and verify the knowledge base is working
1. The knowledge base should have been created by now. open it on the Bedrock AWS console.
2. click the data source and click [sync]. Wait for the sync to complete.
3. Verify that the sync history shows 1 under [source files] column.
4. Use the left pain to sanity test the knowledge base by asking a basic question.

## Let's run the evaluation
We'll want to test how different models for RAG Generation perform. You can test other variations as well, besides generation model, like the prompt template used for generation, etc.
1. In the console navigate to [Amazon Bedrock] > [Evaluations] > [RAG] > [Create].
2. Give the evaluation job a name that will help you make sense of it, for example: `eval-kb-bituach-nova-pro`, or `eval-kb-bituach-nova-lite`.
3. As an [Evaluator model] choose: Sonnet 3.5.
4. Choose the knowledge base we previously created.
5. Choose [RAG evaluation type] choose: `Retrieval and response generation` (this corresponds to the jsonl we generated).
6. [Response generator model] select the model we want to test. For example [Nova Pro].
7. [Metrics]: to save time, choose only `Correctness` and `Completeness`.
8. [Dataset and evaluation results S3 location]: For [Dataset for evaluation] enter the S3 URI for the eval object you uploaded previously.
9  [Dataset and evaluation results S3 location]: For [Results for evaluation] Use something like `s3://my-bucket/bituah-leumi/eval/results/`.
10. [IAM Permissions]: choose `Create and use a new service role`.
11. Click [Create].
12. The new job should appear with an `in progress` Status.
13. Repeat this a second time with a different choice for a generation model.

## Compare evaluations
Once both Evaluation jobs switched to a `Completed` status, you can examine each, or compare both.